# Energy Reconstruction Using CNN - Both Charges and Cos(Zenith)

In [1]:
import numpy as np
import os
import time
from tensorflow import keras
from keras import layers
from keras import models
from keras import callbacks
from keras.callbacks import CSVLogger, EarlyStopping
from data_tools import load_preprocessed, dataPrep, filterReco

## Model Design

In [2]:
# Set the number of epochs the model should run for 
numepochs = 3

# Name for model
name = 'test_filter'

# Baseline data prep
prep = {'q':None, 't':False, 'normed':True, 'reco':'plane', 'cosz':False}

In [3]:
# Add identifying number to name
i = 0
# Saves the h5 file of the model in a folder named models 
while(os.path.exists('models/{}.h5'.format(name+str(i)))): 
    i += 1
name += str(i)
print(name)

test_filter1


In [4]:
# Create model using functional API for multiple inputs

# Input layer 
charge_input = keras.Input(shape=(10,10,2,), name='charge')

# Starts off with three convolutional layers, each one has half the neurons of the previous one 
conv1_layer = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')(charge_input)
conv2_layer = layers.Conv2D(32, kernel_size=3, padding='same', activation='relu')(conv1_layer)
conv3_layer = layers.Conv2D(16, kernel_size=3, padding='same', activation="relu")(conv2_layer)

# Layers are flattened before Zenith information is added 
flat_layer = layers.Flatten()(conv3_layer)
zenith_input = keras.Input(shape=(1,), name='zenith')
concat_layer = layers.Concatenate()([flat_layer, zenith_input])

# The flattened layers and the Zenith layer run through 3 dense layers
dense1_layer = layers.Dense(256, activation='relu')(concat_layer)
dense2_layer = layers.Dense(256, activation='relu')(dense1_layer)
dense3_layer = layers.Dense(256, activation="relu")(dense2_layer)

# This last dense layer is the output of the model
output = layers.Dense(1)(dense3_layer)

model = models.Model(inputs=[charge_input, zenith_input], outputs=output, name=name)
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae','mse'])

In [5]:
model.summary()

Model: "test_filter1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 charge (InputLayer)            [(None, 10, 10, 2)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 10, 10, 64)   1216        ['charge[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 10, 10, 32)   18464       ['conv2d[0][0]']                 
                                                                                                  
 conv2d_2 (Conv2D)              (None, 10, 10, 16)   4624        ['conv2d_1[0][0]']               
                                                                                       

In [6]:
# File directory to folder that holds simulation data 
simPrefix = os.getcwd()+'\\simdata'

# Load simulation data from files for training
x, y = load_preprocessed(simPrefix, 'train')

Percentage of events with a NaN: 2.68


In [7]:
# Prepare simulation data
x_i = dataPrep(x, y, **prep)

# Filter NaNs from reconstruction data
filterReco(prep, y, x_i)

In [11]:
# Training

csv_logger = CSVLogger('models/{}'.format(name))
# Earlystoping stops the model from training when it starts to overfit to the data
# The main parameter we change is the patience 
early_stop = EarlyStopping(monitor="val_loss", min_delta=0, patience=10, verbose=0, mode="auto", baseline=None, restore_best_weights=False) 
callbacks = [early_stop, csv_logger]

history = model.fit({"charge":x_i[0], "zenith":x_i[1].reshape(-1,1)}, y=y['energy'], epochs=numepochs, validation_split=0.15, callbacks=callbacks)

Epoch 1/3
14354/14354 [==============================] - 214s 15ms/step - loss: 0.0866 - mae: 0.1851 - mse: 0.0866 - val_loss: 0.0314 - val_mae: 0.1387 - val_mse: 0.0314
Epoch 2/3
14354/14354 [==============================] - 211s 15ms/step - loss: 0.0317 - mae: 0.1335 - mse: 0.0317 - val_loss: 0.0245 - val_mae: 0.1164 - val_mse: 0.0245
Epoch 3/3
14354/14354 [==============================] - 182s 13ms/step - loss: 0.0284 - mae: 0.1254 - mse: 0.0284 - val_loss: 0.0223 - val_mae: 0.1085 - val_mse: 0.0223


In [12]:
# Save the model results as a .npy and .h5 file
model.save('models/%s.h5' % name)
np.save('models/%s.npy' % name, prep)

# Open a .txt file and write the results of the last epoch
f = open("results.txt", "a")
f.write("{}\nName: {}\nEpochs: {}\nLoss (mse): {}\nVal_loss (mse): {}\n\n".format(
    time.strftime("%m/%d/%Y %H:%M:%S", time.localtime()),
    name,
    len(history.history['loss']),
    history.history['loss'][len(history.history['loss'])-1],
    history.history['val_loss'][len(history.history['loss'])-1]
))
f.close()